# IFU / IWU 

20241010 

1. 전체샷에 대해  unique_id, TEST별로 X_reg_avg 계산.  이걸 M3S
2. Patial shot은 제거하고, Full shot에서만 test no별 avg계산해서 ifu 계산




In [4]:
import pandas as pd
import numpy as np

# 데이터 불러오기
file_path = 'RawData-1.csv'
df = pd.read_csv(file_path)

# Wafer 반경 설정
wafer_radius = 150000

# 각 unique_id 그룹별로 partial 여부 확인
partial_shot_info = []

# 그룹화하여 step pitch 값을 사용
for unique_id, group in df.groupby('UNIQUE_ID'):
    step_pitch_x_value = group['STEP_PITCH_X'].iloc[0]
    step_pitch_y_value = group['STEP_PITCH_Y'].iloc[0]
    
    for _, row in group.iterrows():
        dx, dy = row['DieX'], row['DieY']
        x, y = row['fcp_x'], row['fcp_y']
        
        bottom_left = (x - step_pitch_x_value / 2, y - step_pitch_y_value / 2)
        bottom_right = (x + step_pitch_x_value / 2, y - step_pitch_y_value / 2)
        top_left = (x - step_pitch_x_value / 2, y + step_pitch_y_value / 2)
        top_right = (x + step_pitch_x_value / 2, y + step_pitch_y_value / 2)
        
        corners = [bottom_left, bottom_right, top_left, top_right]
        distances = [np.sqrt(c[0]**2 + c[1]**2) for c in corners]
        
        is_partial = any(dist > wafer_radius for dist in distances)
        
        shot_info = {
            "Unique_ID": unique_id,
            "Die_X": dx, 
            "Die_Y": dy,
            "Is Partial Shot": is_partial,
            "Bottom Left": bottom_left,
            "Bottom Right": bottom_right,
            "Top Left": top_left,
            "Top Right": top_right
        }
        
        partial_shot_info.append(shot_info)

# partial_shot_info 리스트를 DataFrame으로 변환
partial_shot_df = pd.DataFrame(partial_shot_info)

# 기존 DataFrame과 Is Partial Shot 열을 병합
df_merged = pd.concat([df, partial_shot_df['Is Partial Shot']], axis=1)

df_merged.to_csv('df_merged_avg_fullshot만.csv', index=False)

# Partial Shot이 아닌 데이터만 필터링
df_non_partial = df_merged[df_merged['Is Partial Shot'] == False]

# ifu_x와 ifu_y 계산 - Partial Shot을 제외하고 각 unique_id와 test no별로 X_reg, Y_reg의 평균 계산
df_avg = df_non_partial.groupby(['UNIQUE_ID', 'TEST']).agg({'X_reg': 'mean', 'Y_reg': 'mean'}).reset_index()
df_avg.rename(columns={'X_reg': 'X_reg_avg', 'Y_reg': 'Y_reg_avg'}, inplace=True)


df_avg.to_csv('df_avg_fullshot만.csv', index=False)


# 각 unique_id별로 ifu_x와 ifu_y 계산
ifu_values = []
for unique_id, group in df_avg.groupby('UNIQUE_ID'):
    mean_x = np.mean(group['X_reg_avg'])
    std_dev_x = np.std(group['X_reg_avg'])
    ifu_x = abs(mean_x) + (3 * std_dev_x)
    
    mean_y = np.mean(group['Y_reg_avg'])
    std_dev_y = np.std(group['Y_reg_avg'])
    ifu_y = abs(mean_y) + (3 * std_dev_y)
    
    ifu_values.append({'UNIQUE_ID': unique_id, 'ifu_x': ifu_x, 'ifu_y': ifu_y})

# ifu_x와 ifu_y 결과를 DataFrame으로 변환
df_ifu = pd.DataFrame(ifu_values)

# 결과 확인
df_ifu



,UNIQUE_ID,ifu_x,ifu_y
0,VH075030_PTVB827_3GJPBVH.VH075P_PTXV821_2024-0...,0.001692,0.002208
1,WC046030_PTXV831_54BPPWC.WC046P_DBF_PTVP841_20...,0.001233,0.001577
2,WF075030_PTVP841_5G4PPWF.WF075P_-_2024-07-16 2...,0.001616,0.001756
